<a href="https://colab.research.google.com/gist/zitterbewegung/bf7fc4a7921f839b370e63a5dda92c1d/trump-no-retweets-stop-and-end-gpt-2-text-generating-model-w-gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: May 19th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory now uses an Nvidia T4 GPU, which is slightly faster than the old Nvidia K80 GPU for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Mon Aug  5 22:58:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are two sizes of GPT-2:

* `117M` (default): the "small" model, 500MB on disk.
* `345M`: the "medium" model, 1.5GB on disk.

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="345M")

Fetching checkpoint: 1.05Mit [00:00, 219Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 97.2Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 264Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:08, 164Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 239Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 88.8Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 147Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "/content/trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-as-link.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:

gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='345M',
              steps=4000,
              restore_from='latest',
              run_name='trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt',
              print_every=100,
              sample_every=200,
              save_every=1000,
              overwrite=True
              )

W0713 11:53:41.719954 139994155657088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:164: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0713 11:53:41.723483 139994155657088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0713 11:53:56.185151 139994155657088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/sample.py:71: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0713 11:53:56.209380 139994155657088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/sample.py:17: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future versi

Loading checkpoint models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.22s/it]


dataset has 1104119 tokens
Training...
Saving checkpoint/trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt/model-0
[100 | 325.31] loss=2.03 avg=2.03
[200 | 638.27] loss=1.65 avg=1.84
======== SAMPLE 1 ========
 who are at odds with Trump. He is an embarrassment to our country. It does not make any sense!<|endoftext|>
<|startoftext|>With so much corruption in Washington we do not know how much of the enormous &amp; very high cost of healthcare is going to be paid for by the government. I will fix!<|endoftext|>
<|startoftext|>There was a big win by the Democrats in the Senate. The Freedom Caucus gave everything for the good of the Freedom Caucus. No way to beat them in the Senate.<|endoftext|>
<|startoftext|>Thank you Virginia for your hard work. A great and special day in Richmond!<|endoftext|>
<|startoftext|>Thank you for the kind words and kind words from both sides of the debate in the debate last night. Enjoy!<|endoftext|>
<|startoftext|>If ever there was a deb

W0713 12:50:01.319637 139994155657088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========
>A lot of great and loyal folks out there who are going to help get me reelected. Many of those people know me personally from my various ventures. This is a total REPUBLICAN BLOCK. They will have their own ideas and will be very vocal!<|endoftext|>
<|startoftext|>My great honor to hold this year’s “Statesman of the Year” Inaugural Ball at the Old Post Office Pavilion in Washington D.C. URL #MAGA URL<|endoftext|>
<|startoftext|>Join me in Phoenix tomorrow at 11:30am! My wonderful wife is honored to be hosting the 2018 @USOpen!URL<|endoftext|>
<|startoftext|>Just landed in Phoenix a very special evening - a wonderful honor! Thank you Phoenix!#Trump2016 URL<|endoftext|>
<|startoftext|>I will be making a major speech in Phoenix Ohio at 11:30am - it will be fantastic! This is a MOVEMENT like no other. Thank you! #MAGA URL<|endoftext|>
<|startoftext|>I will be in Phoenix at the Old Post Office Pavilion in D.C. next Saturday at the 11:30am start time. Great crowd! 

KeyboardInterrupt: ignored

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt')

W0728 00:30:30.897094 140544924612480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:90: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0728 00:30:30.899383 140544924612480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:100: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0728 00:30:32.196897 140544924612480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:340: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 00:30:32.214372 140544924612480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0728 00:30:37.458852 140544924612480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/g

Loading checkpoint checkpoint/trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:

text = gpt2.generate(sess, run_name='trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt',length=140,truncate='<|endoftext|>',include_prefix=False,prefix='<|startoftext|>', nsamples=2000, return_as_list=True, batch_size=20)

In [0]:
df = pd.DataFrame({'col':text})
df.head()
df['class']='0'
df.head()

,col,class
0,Just left the @WhiteHouse. Amazing journey of ...,0
1,The @nyjets are getting good play from their d...,0
2,@Natalie_Grund Fear not we will have a movie s...,0
3,I hope people stop and think about what is goi...,0
4,“@ApprenticeNBC Don't be afraid of mistakes. U...,0


In [0]:
df2 = pd.read_fwf("/content/trump-tagged-no-retweets-06-14-2019-delimitersremovednolinks.txt")
df2['class']='1'
df2.head()
spam_ham_trump = df.append(df2)
#spam_ham_trump.head()
spam_ham_trump.describe()


,col,class
count,28318,28318
unique,28124,2
top,MAKE AMERICA GREAT AGAIN!,1
freq,27,26318


In [0]:
dfimport numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
%matplotlib inline

In [0]:
!pip install spacy
import spacy

In [0]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()
  
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [0]:
from sklearn.model_selection import train_test_split

X = spam_ham_trump["col"]
ylabels = spam_ham_trump["class"]

#X = df_amazon['verified_reviews'] # the features we want to analyze
#ylabels = df_amazon['feedback'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [0]:
# Logistic Regression Classifier
#from sklearn.linear_model import LogisticRegression
#classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
#from sklearn.svm import SVC
#clf = SVC(gamma='auto')
clf.fit(X, y) 
pipe.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7fd0dbdb18d0>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7fd0774a78c8>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
            

In [0]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
#print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
#print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.94515065913371


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1243: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: ignored

In [0]:
import json
f = open('twillio-load-trump-tagged-no-retweets-06-14-2019-delimitersadded.txt', 'w')
json.dump(text, f)
f.close()
  

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )

LORD FITZWATER:
Saw'st thou not, sir?

QUEEN ELIZABETH:
Madam, didst thou not mean, as some do,
To cram others with beggary?

QUEEN ELIZABETH:
Ha, ha! what should you say? why didst thou say no?

QUEEN ELIZABETH:
Hath this woman in this world hungrier than man?
Think'st thou that all these woes can be avoided?

QUEEN ELIZABETH:
As many lives she hath prevented as man.

DUCHESS OF YORK:
Why, hark! the Tower is brought down.

QUEEN ELIZABETH:
The King of Naples comes apace.

DUCHESS OF YORK:
It shall be so, I hope.

QUEEN ELIZABETH:
The Tower comes down.

DUCHESS OF YORK:
The Naples comes apace.

QUEEN ELIZABETH:
It shall be so, I hope.

DUCHESS OF YORK:


LORDY:
Every man that's gone hath died
And that's the end of all.

GLOUCESTER:
Your lordship must be revenged, Hermione.

LADY ANNE:
And she shall be the man that I dreamt of.

GLOUCESTER:
I cannot sleep: never a waking man shall
Go to any sleepder than to sleep.

LADY ANNE:
It shall be such a dream, I'll bear witness to it,
That I may

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp and then download it.

You can rerun the cell as many times as you want for even more generated texts!

In [0]:
,length=140,truncate='<|endoftext|>',include_prefix=False,prefix='<|startoftext|>', nsamples=2000, return_as_list=True, batch_size=20

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=140,
                      temperature=0.9,
                      nsamples=2000,
                      batch_size=20,
                      truncate='<|endoftext|>',
                      include_prefix=False,
                      prefix='<|startoftext|>',
                      run_name='trump-tagged-no-retweets-06-14-2019-delimitersadded-symbol-for-url2.txt',
                      top_k=40
                      )

files.download(gen_file)

W0713 14:54:25.650161 140501625440128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/sample.py:71: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0713 14:54:25.677207 140501625440128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/sample.py:17: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0713 14:54:25.682593 140501625440128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/sample.py:77: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


# Etcetera

If the notebook has errors (e.g. GPU Sync Fail or out-of-memory/OOM), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

In [0]:
import pandas
df = pandas.read_csv("/content/subset.csv",header=0)
df['text']


,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,Today’s shooting in El Paso Texas was not only...,08-04-2019 04:19:01,25799,146075,False,1157868518823596032
1,Twitter for iPhone,....Melania and I send our heartfelt thoughts ...,08-04-2019 04:19:01,15893,108001,False,1157868519964499968
2,Twitter for iPhone,The people on the stage tonight and last were ...,08-01-2019 04:05:15,28886,144147,False,1156777889100173313


In [0]:
df['text']

0    Today’s shooting in El Paso Texas was not only...
1    ....Melania and I send our heartfelt thoughts ...
2    The people on the stage tonight and last were ...
Name: text, dtype: object

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.